In [19]:
#import required libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [25]:
#which divisions and years would we like to pull
divisionList = [1,2] 
yearList = [12,13]

# 2015

In [26]:
#starting url for scraping
start = "https://games.crossfit.com/scores/leaderboard.php?stage=5&sort=0&division=%d&region=%d&regional=5&numberperpage=60&page=%d&competition=0&frontpage=0&expanded=1&full=1&year=%d&showtoggles=0&hidedropdowns=1&showathleteac=1&athletename=&fittest=1&fitSelect=0&scaled=0&occupation=0"

In [27]:
#Create empty list
df_list = []
#iterate through all the pages for each region, division, year and scrape leaderboard tables
for year in yearList:
    for division in divisionList:
        for region in range(1,18): #there are 17 distinct regions to want to iterate up until 18
            parameters = {"page":0,"division":division,"region":region} #where to start the pull requests
            response = requests.get(start, params=parameters)
            soup = BeautifulSoup(response.text, 'lxml')
    
            mybuttons = soup.findAll('a', {'class': 'button'}) # where is the next button for pagination
            if len(mybuttons) > 0:
                numPages = int(mybuttons[0].text)
            else:
                numPages = 1
            for page in range(1, numPages+1): #this is how we iterate through all the pages for each region
                #assigning some values
                c = {'year': '%d' %(year), 'competitionType': 'open', 'regionName': '%d' %(region), 'division': '%d' %(division)}
                url_head = start %(division,region,page,year)
                #for each region, division, year appending the data to a list
                df_list.append(pd.read_html(url_head, header=0, encoding ='UTF-8')[0].rename({'Unnamed: 0':'RegionRank'}, axis = 'columns').assign(**c)) 
df = pd.concat(df_list) #concatenating list into one dataframe


C:\Users\jess\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [28]:
#Checking that dataframe looks good
df.head()

,Competitor,RegionRank,Unnamed: 7,Workout 01,Workout 02,Workout 03,Workout 04,Workout 05,competitionType,division,regionName,year
0,David Levey,1 (27),Validated by:,15 (110) Validated by: CrossFit East London,1 (80) Validated by: CrossFit East London,4 (359) Validated by: CrossFit East London,2 (258) Validated by: CrossFit East London,5 (121) Validated by: CrossFit East London,open,1,1,12
1,Danie du Preez,2 (44),Validated by:,34 (104) Validated by: CrossFit B4C,1 (80) Validated by: CrossFit B4C,2 (379) Validated by: CrossFit B4C,5 (256) Validated by: CrossFit B4C,2 (126) Validated by: CrossFit B4C,open,1,1,12
2,Jason Smith,2 (44),Validated by:,23 (107) Validated by: CrossFit Platinum,6 (73) Validated by: CrossFit Platinum,3 (375) Validated by: CrossFit Platinum,10 (250) Validated by: CrossFit Platinum,2 (126) Validated by: CrossFit Platinum,open,1,1,12
3,Neil Scholtz,4 (52),Validated by:,7 (117) Validated by: Ballistix CrossFit,18 (66) Validated by: Ballistix CrossFit,14 (311) Validated by: Ballistix CrossFit,3 (257) Validated by: Ballistix CrossFit,10 (108) Validated by: Ballistix CrossFit,open,1,1,12
4,Frederik Engelbrecht,5 (65),Validated by:,34 (104) Validated by: CrossFit PBM,9 (70) Validated by: CrossFit PBM,6 (343) Validated by: CrossFit PBM,11 (248) Validated by: CrossFit PBM,5 (121) Validated by: CrossFit PBM,open,1,1,12


In [144]:
#Splitting workout attributes into workout score and rank values
df[['ScoreOneRank','ScoreOne']] = df['Workout  01'].str.split('(',expand=True)
df[['ScoreOneARank','ScoreOneA']] = df['Workout  01A'].str.split('(',expand=True)
df[['ScoreTwoRank','ScoreTwo']] = df['Workout  02'].str.split('(',expand=True)
df[['ScoreThreeRank','ScoreThree']] = df['Workout  03'].str.split('(',expand=True)
df[['ScoreFourRank','ScoreFour']] = df['Workout  04'].str.split('(',expand=True)
df[['ScoreFiveRank','ScoreFive']] = df['Workout  05'].str.split('(',expand=True)
df[['RegionRank','RegionScore']] = df['RegionRank'].str.split('(',expand=True)
df.head()

,RegionRank,Competitor,Workout 01,Workout 01A,Workout 02,Workout 03,Workout 04,Workout 05,yearList,competitionType,...,ScoreOneA,ScoreTwoRank,ScoreTwo,ScoreThreeRank,ScoreThree,ScoreFourRank,ScoreFour,ScoreFiveRank,ScoreFive,RegionScore
0,1,Jason Smith,4 (219),8 (315),9 (270),2 (561),8 (125),3 (06:12),2015,open,...,315),9,270),2,561),8,125),3,06:12),34)
1,2,Mikko Aronpää,5 (217),6 (317),5 (278),18 (489),9 (122),5 (06:16),2015,open,...,317),5,278),18,489),9,122),5,06:16),48)
2,3,Michael Mikaere,28 (198),29 (297),7 (272),4 (524),2 (144),6 (06:17),2015,open,...,297),7,272),4,524),2,144),6,06:17),76)
3,4,ates boran,8 (209),26 (300),3 (311),16 (493),6 (126),27 (07:09),2015,open,...,300),3,311),16,493),6,126),27,07:09),86)
4,5,Utku Tuncer,20 (203),13 (310),19 (262),17 (490),1 (147),21 (06:53),2015,open,...,310),19,262),17,490),1,147),21,06:53),91)


In [154]:
#Additional cleanup for the split attributes
df['ScoreOne'] = df['ScoreOne'].str.split(')').str[0]
df['ScoreOneA'] = df['ScoreOneA'].str.split(')').str[0]
df['ScoreTwo'] = df['ScoreTwo'].str.split(')').str[0]
df['ScoreThree'] = df['ScoreThree'].str.split(')').str[0]
df['ScoreFour'] = df['ScoreFour'].str.split(')').str[0]
df['ScoreFive'] = df['ScoreFive'].str.split(')').str[0]
df['RegionScore'] = df['RegionScore'].str.split(')').str[0]
df.head()

,RegionRank,Competitor,Workout 01,Workout 01A,Workout 02,Workout 03,Workout 04,Workout 05,yearList,competitionType,...,ScoreOneA,ScoreTwoRank,ScoreTwo,ScoreThreeRank,ScoreThree,ScoreFourRank,ScoreFour,ScoreFiveRank,ScoreFive,RegionScore
0,1,Jason Smith,4 (219),8 (315),9 (270),2 (561),8 (125),3 (06:12),2015,open,...,315,9,270,2,561,8,125,3,06:12,34
1,2,Mikko Aronpää,5 (217),6 (317),5 (278),18 (489),9 (122),5 (06:16),2015,open,...,317,5,278,18,489,9,122,5,06:16,48
2,3,Michael Mikaere,28 (198),29 (297),7 (272),4 (524),2 (144),6 (06:17),2015,open,...,297,7,272,4,524,2,144,6,06:17,76
3,4,ates boran,8 (209),26 (300),3 (311),16 (493),6 (126),27 (07:09),2015,open,...,300,3,311,16,493,6,126,27,07:09,86
4,5,Utku Tuncer,20 (203),13 (310),19 (262),17 (490),1 (147),21 (06:53),2015,open,...,310,19,262,17,490,1,147,21,06:53,91


In [156]:
#dropping extraneous attributes
df.drop(df.iloc[:, 2:8], axis=1, inplace=True)
df.head()

,RegionRank,Competitor,yearList,competitionType,regionName,division,ScoreOneRank,ScoreOne,ScoreOneARank,ScoreOneA,ScoreTwoRank,ScoreTwo,ScoreThreeRank,ScoreThree,ScoreFourRank,ScoreFour,ScoreFiveRank,ScoreFive,RegionScore
0,1,Jason Smith,2015,open,1,1,4,219,8,315,9,270,2,561,8,125,3,06:12,34
1,2,Mikko Aronpää,2015,open,1,1,5,217,6,317,5,278,18,489,9,122,5,06:16,48
2,3,Michael Mikaere,2015,open,1,1,28,198,29,297,7,272,4,524,2,144,6,06:17,76
3,4,ates boran,2015,open,1,1,8,209,26,300,3,311,16,493,6,126,27,07:09,86
4,5,Utku Tuncer,2015,open,1,1,20,203,13,310,19,262,17,490,1,147,21,06:53,91


In [157]:
len(df)

289624

In [158]:
#save dataframe to csv file
df.to_csv('data2015.csv')

# 2016

In [159]:
start = "https://games.crossfit.com/scores/leaderboard.php?stage=0&sort=0&division=1&region=0&regional=5&numberperpage=60&page=0&competition=0&frontpage=0&expanded=1&full=1&year=16&showtoggles=0&hidedropdowns=1&showathleteac=1&athletename=&fittest=1&fitSelect=0&scaled=0&occupation=0"

In [160]:
divisionList = [1,2] 

In [161]:
df_list = []
for division in divisionList:
    for region in range(1,18):
        parameters = {"page":0,"division":division,"region":region}
        response = requests.get(start, params=parameters)
        soup = BeautifulSoup(response.text, 'lxml')
    
        mybuttons = soup.findAll('a', {'class': 'button'})
        if len(mybuttons) > 0:
            numPages = int(mybuttons[0].text)
        else:
            numPages = 1
        for page in range(1, numPages+1):
            c = {'yearList': '2016', 'competitionType': 'open', 'regionName': '%d' %(region), 'division': '%d' %(division)}
            url_head = "https://games.crossfit.com/scores/leaderboard.php?stage=0&sort=0&division=%d&region=%d&regional=5&numberperpage=60&page=%d&competition=0&frontpage=0&expanded=1&full=1&year=16&showtoggles=0&hidedropdowns=1&showathleteac=1&athletename=&fittest=1&fitSelect=1&scaled=0&occupation=0" %(division,region,page)
            df_list.append(pd.read_html(url_head, header=0, encoding ='UTF-8')[0].rename({'Unnamed: 0':'RegionRank'}, axis = 'columns').assign(**c)) 
df = pd.concat(df_list)


In [162]:
df.head()

,RegionRank,Competitor,Workout 01,Workout 02,Workout 03,Workout 04,Workout 05,yearList,competitionType,regionName,division
0,1 (19),Jason Smith,8 (297),3 (428),5 (130),1 (277),2 (08:55),2016,open,1,1
1,2 (29),Ruan Duvenage,5 (310),2 (429),11 (127),7 (268),4 (08:57),2016,open,1,1
2,3 (38),Mikko Aronpää,6 (305),6 (424),1 (138),8 (266),17 (09:27),2016,open,1,1
3,4 (46),Phil Hesketh,2 (312),1 (430),21 (121),21 (250),1 (08:41),2016,open,1,1
4,5 (51),Billy Pullen,1 (313),17 (345),2 (134),29 (244),2 (08:55),2016,open,1,1


In [163]:
df[['ScoreOneRank','ScoreOne']] = df['Workout  01'].str.split('(',expand=True)
df[['ScoreTwoRank','ScoreTwo']] = df['Workout  02'].str.split('(',expand=True)
df[['ScoreThreeRank','ScoreThree']] = df['Workout  03'].str.split('(',expand=True)
df[['ScoreFourRank','ScoreFour']] = df['Workout  04'].str.split('(',expand=True)
df[['ScoreFiveRank','ScoreFive']] = df['Workout  05'].str.split('(',expand=True)
df[['RegionRank','RegionScore']] = df['RegionRank'].str.split('(',expand=True)
df.head()

,RegionRank,Competitor,Workout 01,Workout 02,Workout 03,Workout 04,Workout 05,yearList,competitionType,regionName,...,ScoreOne,ScoreTwoRank,ScoreTwo,ScoreThreeRank,ScoreThree,ScoreFourRank,ScoreFour,ScoreFiveRank,ScoreFive,RegionScore
0,1,Jason Smith,8 (297),3 (428),5 (130),1 (277),2 (08:55),2016,open,1,...,297),3,428),5,130),1,277),2,08:55),19)
1,2,Ruan Duvenage,5 (310),2 (429),11 (127),7 (268),4 (08:57),2016,open,1,...,310),2,429),11,127),7,268),4,08:57),29)
2,3,Mikko Aronpää,6 (305),6 (424),1 (138),8 (266),17 (09:27),2016,open,1,...,305),6,424),1,138),8,266),17,09:27),38)
3,4,Phil Hesketh,2 (312),1 (430),21 (121),21 (250),1 (08:41),2016,open,1,...,312),1,430),21,121),21,250),1,08:41),46)
4,5,Billy Pullen,1 (313),17 (345),2 (134),29 (244),2 (08:55),2016,open,1,...,313),17,345),2,134),29,244),2,08:55),51)


In [164]:
df['ScoreOne'] = df['ScoreOne'].str.split(')').str[0]
df['ScoreTwo'] = df['ScoreTwo'].str.split(')').str[0]
df['ScoreThree'] = df['ScoreThree'].str.split(')').str[0]
df['ScoreFour'] = df['ScoreFour'].str.split(')').str[0]
df['ScoreFive'] = df['ScoreFive'].str.split(')').str[0]
df['RegionScore'] = df['RegionScore'].str.split(')').str[0]
df.head()

,RegionRank,Competitor,Workout 01,Workout 02,Workout 03,Workout 04,Workout 05,yearList,competitionType,regionName,...,ScoreOne,ScoreTwoRank,ScoreTwo,ScoreThreeRank,ScoreThree,ScoreFourRank,ScoreFour,ScoreFiveRank,ScoreFive,RegionScore
0,1,Jason Smith,8 (297),3 (428),5 (130),1 (277),2 (08:55),2016,open,1,...,297,3,428,5,130,1,277,2,08:55,19
1,2,Ruan Duvenage,5 (310),2 (429),11 (127),7 (268),4 (08:57),2016,open,1,...,310,2,429,11,127,7,268,4,08:57,29
2,3,Mikko Aronpää,6 (305),6 (424),1 (138),8 (266),17 (09:27),2016,open,1,...,305,6,424,1,138,8,266,17,09:27,38
3,4,Phil Hesketh,2 (312),1 (430),21 (121),21 (250),1 (08:41),2016,open,1,...,312,1,430,21,121,21,250,1,08:41,46
4,5,Billy Pullen,1 (313),17 (345),2 (134),29 (244),2 (08:55),2016,open,1,...,313,17,345,2,134,29,244,2,08:55,51


In [165]:
df.drop(df.iloc[:, 2:7], axis=1, inplace=True)
df.head()

,RegionRank,Competitor,yearList,competitionType,regionName,division,ScoreOneRank,ScoreOne,ScoreTwoRank,ScoreTwo,ScoreThreeRank,ScoreThree,ScoreFourRank,ScoreFour,ScoreFiveRank,ScoreFive,RegionScore
0,1,Jason Smith,2016,open,1,1,8,297,3,428,5,130,1,277,2,08:55,19
1,2,Ruan Duvenage,2016,open,1,1,5,310,2,429,11,127,7,268,4,08:57,29
2,3,Mikko Aronpää,2016,open,1,1,6,305,6,424,1,138,8,266,17,09:27,38
3,4,Phil Hesketh,2016,open,1,1,2,312,1,430,21,121,21,250,1,08:41,46
4,5,Billy Pullen,2016,open,1,1,1,313,17,345,2,134,29,244,2,08:55,51


In [166]:
df.to_csv('data2016.csv')